# Client Targeting

* **Section 0: Load dataset**
* **Section 1: Prepare datasets**
  * Data Split into (1) Training and (2) Client Targetting sets
  * Apply data processing
  * Prepare training datasets - further split training data set into (1) train and (2) validation sets
* **Section 2: Models training**
  * Revenue regression models
  * Sales classification models
* **Section 3: Clients targeting**
  * Propensity scoring
  * Predict revenues
  * Prepare list of clients to target

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from utlis.data_utils import load_data, merge_data, process_features1, process_features2, get_feature_cols
from utlis.model_utils import train_revenue_model_xgb_optuna, train_sales_model_xgb_optuna_f1, predict_propensity
from utlis.targeting import calculate_revenues, run_full_targeting_pipeline, print_targeting_summary, assign_best_offer


ImportError: cannot import name 'train_sales_model_xgb_optuna' from 'utlis.model_utils' (c:\Users\edang\OneDrive\Maven\test1\utlis\model_utils.py)

## Section 0: Load dataset

In [ ]:
print("1. Loading data...")
file = 'DataScientist_CaseStudy_Dataset.xlsx'
soc_dem, products, inflow, sales = load_data(file)
df = merge_data(soc_dem, products, inflow, sales)


1. Loading data...


## Section 1: Prepare datasets

### Data Split into (1) Training and (2) Client Targetting sets

In [ ]:
classification_target_columns = ['Sale_CL', 'Sale_CC', 'Sale_MF']
regression_target_columns = ['Revenue_CL','Revenue_CC','Revenue_MF']

# Training data set
train_val = df.dropna(subset=classification_target_columns+regression_target_columns, how='all')

# Client Targetting set
test = df[df[classification_target_columns+regression_target_columns].isna().all(axis=1)].copy()

print(f"Training set: {train_val.shape[0]} clients ({train_val.shape[0]/len(df)*100:.1f}%)")
print(f"Client Targetting set: {test.shape[0]} clients ({test.shape[0]/len(df)*100:.1f}%)")

Training set: 969 clients (60.0%)
Client Targetting set: 646 clients (40.0%)


### Apply data processing

In [ ]:
print( "*"*30 + "Before train_val processing" + "*"*30 + "\n")
print( train_val.head(3))
print( f"\n{train_val.shape=}")
train_val, sex_label_encoder = process_features2(train_val)
print("\n")

print( "*"*30 + "Before test processing" + "*"*30 + "\n")
print( test.head(3))
print( f"\n{test.shape=}")
test, _ = process_features2(test, le=sex_label_encoder)

******************************Before train_val processing******************************

   Client Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  Count_CC  \
1    1217   M   38     165         1       NaN       NaN        NaN       NaN   
2     850   F   49      44         1       NaN       NaN        NaN       NaN   
3    1473   M   54      34         1       1.0       NaN        NaN       1.0   

   Count_CL  ...  TransactionsDeb_CA  TransactionsDebCash_Card  \
1       NaN  ...                 1.0                       0.0   
2       NaN  ...                 6.0                       0.0   
3       1.0  ...                38.0                       1.0   

   TransactionsDebCashless_Card  TransactionsDeb_PaymentOrder  Sale_MF  \
1                           0.0                           1.0      0.0   
2                           0.0                           1.0      0.0   
3                          26.0                          11.0      1.0   

   Sale_CC  Sale_CL  Rev

In [ ]:
print( "*"*30 + "After train_val processing" + "*"*30 + "\n")
print( train_val.head(3))
print( f"\n{train_val.shape=}")
print("\n")

print( "*"*30 + "After test processing" + "*"*30 + "\n")
print( test.head(3))
print( f"\n{test.shape=}")

******************************After train_val processing******************************

   Client  Sex  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  \
1    1217    1   38     165         1       NaN       NaN        NaN   
2     850    0   49      44         1       NaN       NaN        NaN   
3    1473    1   54      34         1       1.0       NaN        NaN   

   Count_CC  Count_CL  ...  TransactionsDebCash_Card  \
1       NaN       NaN  ...                       0.0   
2       NaN       NaN  ...                       0.0   
3       1.0       1.0  ...                       1.0   

   TransactionsDebCashless_Card  TransactionsDeb_PaymentOrder  Sale_MF  \
1                           0.0                           1.0      0.0   
2                           0.0                           1.0      0.0   
3                          26.0                          11.0      1.0   

   Sale_CC  Sale_CL  Revenue_MF  Revenue_CC  Revenue_CL  VolumeCredDebRatio  
1      0.0      0.0    

In [ ]:
feature_cols = get_feature_cols(train_val)
train_val[feature_cols].isnull().sum()

Sex                               0
Age                               0
Tenure                            0
Count_CA                          0
Count_SA                        703
Count_MF                        783
Count_OVD                       716
Count_CC                        857
Count_CL                        888
ActBal_CA                         0
ActBal_SA                       703
ActBal_MF                       783
ActBal_OVD                      716
ActBal_CC                       857
ActBal_CL                       888
VolumeCred_CA                     0
TransactionsCred                  0
VolumeDeb                         0
VolumeDebCash_Card                0
VolumeDebCashless_Card            0
VolumeDeb_PaymentOrder            0
TransactionsDeb                   0
TransactionsDebCash_Card          0
TransactionsDebCashless_Card      0
TransactionsDeb_PaymentOrder      0
VolumeCredDebRatio                0
dtype: int64

In [ ]:
target_columns = classification_target_columns + regression_target_columns
train_val[target_columns].isnull().sum()

Sale_CL       0
Sale_CC       0
Sale_MF       0
Revenue_CL    0
Revenue_CC    0
Revenue_MF    0
dtype: int64

In [ ]:
test[feature_cols].isnull().sum()

Sex                               0
Age                               0
Tenure                            0
Count_CA                          0
Count_SA                        484
Count_MF                        523
Count_OVD                       477
Count_CC                        585
Count_CL                        589
ActBal_CA                         0
ActBal_SA                       484
ActBal_MF                       523
ActBal_OVD                      477
ActBal_CC                       585
ActBal_CL                       589
VolumeCred_CA                     0
TransactionsCred                  0
VolumeDeb                         0
VolumeDebCash_Card                0
VolumeDebCashless_Card            0
VolumeDeb_PaymentOrder            0
TransactionsDeb                   0
TransactionsDebCash_Card          0
TransactionsDebCashless_Card      0
TransactionsDeb_PaymentOrder      0
VolumeCredDebRatio                0
dtype: int64

In [ ]:
X_train_val = train_val[feature_cols].fillna(0)
# y_train_val = (train_val[target_columns] > 0).astype(int)
y_train_val = train_val[target_columns]

random_state=42
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=random_state)

print( "X_train.shape, X_val.shape\n", X_train.shape, X_val.shape )

X_train.head(3)


X_train.shape, X_val.shape
 (870, 26) (97, 26)


,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,TransactionsCred,VolumeDeb,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,VolumeCredDebRatio
382,0,48,37,1,0.0,0.0,0.0,0.0,0.0,2262.178929,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
276,0,52,115,1,0.0,0.0,0.0,0.0,0.0,1704.711786,...,3.0,556.364286,125.0,358.042857,41.678571,14.0,2.0,7.0,2.0,0.961160
938,1,34,237,1,1.0,0.0,0.0,0.0,0.0,1161.505714,...,10.0,8567.436429,0.0,84.909643,8477.500000,14.0,0.0,1.0,6.0,0.869058


In [ ]:
print( "y_train.shape, y_val.shape\n", y_train.shape, y_val.shape )
y_train.head(3)

y_train.shape, y_val.shape
 (870, 6) (97, 6)


,Sale_CL,Sale_CC,Sale_MF,Revenue_CL,Revenue_CC,Revenue_MF
382,0.0,1.0,0.0,0.000000,4.035714,0.000000
276,1.0,1.0,0.0,12.208214,4.928571,0.000000
938,1.0,1.0,1.0,133.275357,2.679286,0.368214


## Section 2: Models Training

### Revenue regression models

In [ ]:
models = {}
r2_scores = {}
rmse_scores = {}

# Separate revenue regression models for each product
for product in ['CL', 'CC', 'MF']:
    target_col = f'Revenue_{product}'
    model, r2, rmse, best_params, study = train_revenue_model_xgb_optuna(X_train, X_val, y_train[target_col], y_val[target_col])

    models[f"{product}_revenue"] = model
    r2_scores[product] = r2
    rmse_scores[product] = rmse
    print(f"{product} Revenue Model - R²: {r2:.3f}, RMSE: {rmse:.3f}")



[I 2025-07-14 16:59:05,569] A new study created in memory with name: no-name-83105b15-4c27-49d4-a06b-c775c0b7c430
[I 2025-07-14 16:59:05,904] Trial 0 finished with value: -0.6070059634521843 and parameters: {'n_estimators': 147, 'max_depth': 6, 'learning_rate': 0.12488447489279408, 'subsample': 0.6013876341176071, 'colsample_bytree': 0.8974965636924639, 'reg_alpha': 0.08674394633063215, 'reg_lambda': 1.9670665654984798, 'min_child_weight': 9, 'gamma': 0.3097976400125435}. Best is trial 0 with value: -0.6070059634521843.
[I 2025-07-14 16:59:06,331] Trial 1 finished with value: -0.55551241865081 and parameters: {'n_estimators': 367, 'max_depth': 9, 'learning_rate': 0.11403064544741534, 'subsample': 0.9445166303897705, 'colsample_bytree': 0.7223389175376945, 'reg_alpha': 0.09168842825925608, 'reg_lambda': 0.0015723198786619349, 'min_child_weight': 10, 'gamma': 2.1143707697838936}. Best is trial 1 with value: -0.55551241865081.
[I 2025-07-14 16:59:06,542] Trial 2 finished with value: -0.65

CL Revenue Model - R²: -0.103, RMSE: 4.725


[I 2025-07-14 16:59:34,728] Trial 1 finished with value: -5.915764125328 and parameters: {'n_estimators': 396, 'max_depth': 9, 'learning_rate': 0.017452178855650827, 'subsample': 0.878323582638828, 'colsample_bytree': 0.6841123502927351, 'reg_alpha': 0.18023089565591743, 'reg_lambda': 0.1810625683527769, 'min_child_weight': 1, 'gamma': 1.3294148039620468}. Best is trial 0 with value: -0.7123968648663062.
[I 2025-07-14 16:59:35,223] Trial 2 finished with value: -5.024170693653555 and parameters: {'n_estimators': 356, 'max_depth': 9, 'learning_rate': 0.04843917517070732, 'subsample': 0.9045036687014271, 'colsample_bytree': 0.6054992678512301, 'reg_alpha': 0.008352407815472232, 'reg_lambda': 0.14616733541586088, 'min_child_weight': 2, 'gamma': 4.731691347728141}. Best is trial 0 with value: -0.7123968648663062.
[I 2025-07-14 16:59:36,093] Trial 3 finished with value: -1.9073839467692277 and parameters: {'n_estimators': 382, 'max_depth': 10, 'learning_rate': 0.011926869786077442, 'subsampl

CC Revenue Model - R²: -0.058, RMSE: 5.117


[I 2025-07-14 17:00:00,635] Trial 0 finished with value: -0.15373684421496003 and parameters: {'n_estimators': 120, 'max_depth': 7, 'learning_rate': 0.04678559505478226, 'subsample': 0.9269518118809108, 'colsample_bytree': 0.7759643559593398, 'reg_alpha': 3.1176654543984115, 'reg_lambda': 0.4731222039305065, 'min_child_weight': 1, 'gamma': 4.417693367797036}. Best is trial 0 with value: -0.15373684421496003.
[I 2025-07-14 17:00:01,483] Trial 1 finished with value: -0.15709233455551752 and parameters: {'n_estimators': 379, 'max_depth': 10, 'learning_rate': 0.03659397618260951, 'subsample': 0.8702844843165829, 'colsample_bytree': 0.9155823597787968, 'reg_alpha': 0.06248052031526374, 'reg_lambda': 0.13740165903637036, 'min_child_weight': 1, 'gamma': 0.4994485007651839}. Best is trial 0 with value: -0.15373684421496003.
[I 2025-07-14 17:00:01,705] Trial 2 finished with value: -0.027579046714694577 and parameters: {'n_estimators': 179, 'max_depth': 4, 'learning_rate': 0.042201789259957635, 

MF Revenue Model - R²: -0.006, RMSE: 8.590


In [ ]:
for product in ['CL', 'CC', 'MF']:
    print(f"{product} Revenue Model - R²: {r2_scores[product]:.3f}, RMSE: {rmse_scores[product]:.3f}")

CL Revenue Model - R²: -0.103, RMSE: 4.725
CC Revenue Model - R²: -0.058, RMSE: 5.117
MF Revenue Model - R²: -0.006, RMSE: 8.590


### Sales classification models

In [ ]:
f1_scores = {}

# Separate sales classification models for each product
for product in ['CL', 'CC', 'MF']:
    target_col = f'Sale_{product}'
    model, f1, roc_auc, best_params, study = train_sales_model_xgb_optuna_f1(X_train, X_val, y_train[target_col], y_val[target_col])
    # model, f1, roc_auc, best_params, study  = train_sales_model_xgb_optuna(X_train, X_val, y_train[target_col], y_val[target_col])

    models[f"{product}_sales"] = model
    f1_scores[product] = f1
    print(f"{product} Sales Model - f1: {f1:.3f}")

[I 2025-07-14 17:00:33,202] A new study created in memory with name: no-name-c1777fba-3d27-4364-956a-2d6bb203ff4a
[I 2025-07-14 17:00:33,423] Trial 0 finished with value: 0.4642857142857143 and parameters: {'n_estimators': 196, 'max_depth': 7, 'learning_rate': 0.21100878857541125, 'subsample': 0.8392913654585594, 'colsample_bytree': 0.7163297553488647, 'reg_alpha': 3.0654354472598913, 'reg_lambda': 0.6332118106475881, 'min_child_weight': 2, 'gamma': 0.8542431545011897}. Best is trial 0 with value: 0.4642857142857143.
[I 2025-07-14 17:00:33,905] Trial 1 finished with value: 0.4406779661016949 and parameters: {'n_estimators': 483, 'max_depth': 3, 'learning_rate': 0.014108134210014622, 'subsample': 0.9483255342652636, 'colsample_bytree': 0.6226379130941594, 'reg_alpha': 0.5012862167708656, 'reg_lambda': 0.45183534694248256, 'min_child_weight': 9, 'gamma': 1.1090182263272386}. Best is trial 0 with value: 0.4642857142857143.
[I 2025-07-14 17:00:34,064] Trial 2 finished with value: 0.4827586

CL Sales Model - f1: 0.526


[I 2025-07-14 17:01:00,612] Trial 0 finished with value: 0.42105263157894735 and parameters: {'n_estimators': 131, 'max_depth': 4, 'learning_rate': 0.08010669695185565, 'subsample': 0.8423172013189886, 'colsample_bytree': 0.8892947420619821, 'reg_alpha': 0.010064625319798248, 'reg_lambda': 5.19340826112628, 'min_child_weight': 6, 'gamma': 0.20260788558221499}. Best is trial 0 with value: 0.42105263157894735.
[I 2025-07-14 17:01:01,169] Trial 1 finished with value: 0.35714285714285715 and parameters: {'n_estimators': 384, 'max_depth': 9, 'learning_rate': 0.29845509857885827, 'subsample': 0.6885183592770392, 'colsample_bytree': 0.6083523322493855, 'reg_alpha': 0.10870132930198928, 'reg_lambda': 0.0016998266408792193, 'min_child_weight': 5, 'gamma': 3.0788284754521804}. Best is trial 0 with value: 0.42105263157894735.
[I 2025-07-14 17:01:01,486] Trial 2 finished with value: 0.4583333333333333 and parameters: {'n_estimators': 124, 'max_depth': 8, 'learning_rate': 0.0286198399216346, 'subsa

CC Sales Model - f1: 0.538


[I 2025-07-14 17:01:38,398] Trial 1 finished with value: 0.2978723404255319 and parameters: {'n_estimators': 325, 'max_depth': 6, 'learning_rate': 0.28054749429883935, 'subsample': 0.8820084092423843, 'colsample_bytree': 0.6378410911372512, 'reg_alpha': 0.010922564483605767, 'reg_lambda': 0.15060436923437492, 'min_child_weight': 2, 'gamma': 4.123954310699643}. Best is trial 1 with value: 0.2978723404255319.
[I 2025-07-14 17:01:39,130] Trial 2 finished with value: 0.2608695652173913 and parameters: {'n_estimators': 480, 'max_depth': 5, 'learning_rate': 0.026382439942591915, 'subsample': 0.9327818369958668, 'colsample_bytree': 0.641720162009284, 'reg_alpha': 0.07551330786199276, 'reg_lambda': 5.900586279075081, 'min_child_weight': 6, 'gamma': 4.197908780048798}. Best is trial 1 with value: 0.2978723404255319.
[I 2025-07-14 17:01:39,662] Trial 3 finished with value: 0.3333333333333333 and parameters: {'n_estimators': 317, 'max_depth': 5, 'learning_rate': 0.015000603827951944, 'subsample':

MF Sales Model - f1: 0.429


In [ ]:
for product in ['CL', 'CC', 'MF']:
    print(f"{product} Sales Model - f1: {f1_scores[product]:.3f}")

CL Sales Model - f1: 0.526
CC Sales Model - f1: 0.538
MF Sales Model - f1: 0.429


In [ ]:
print( models.keys() )

dict_keys(['CL_revenue', 'CC_revenue', 'MF_revenue', 'CL_sales', 'CC_sales', 'MF_sales'])


## Section 3: Clients targeting

### Propensity Scoring

In [ ]:
for product in ['CL', 'CC', 'MF']:
    test[f'p_{product.lower()}'] = predict_propensity(models[f"{product}_sales"] , test, feature_cols)

test[['p_cl', 'p_cc', 'p_mf']]

,p_cl,p_cc,p_mf
0,0.570641,0.347243,0.311113
6,0.693686,0.307230,0.469284
9,0.584893,0.435866,0.357840
10,0.376673,0.303365,0.386479
13,0.419662,0.431762,0.432416
...,...,...,...
1598,0.277170,0.390875,0.328764
1600,0.439519,0.373004,0.292288
1608,0.490280,0.518635,0.373756
1610,0.598041,0.433591,0.346199


### Predict Revenues

In [ ]:
predicted_revenues_df = calculate_revenues(test, models)
predicted_revenues_df.head()


,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,Count_CL,...,Sale_MF,Sale_CC,Sale_CL,Revenue_MF,Revenue_CC,Revenue_CL,VolumeCredDebRatio,p_cl,p_cc,p_mf
0,909,1,21,27,1,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,4.252265,3.781702,3.904984,1.747104,0.570641,0.347243,0.311113
6,699,1,37,175,1,NaN,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,1.599131,3.217432,4.853654,1.560034,0.693686,0.307230,0.469284
9,528,0,19,70,1,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,4.163760,4.902122,4.382651,1.114116,0.584893,0.435866,0.357840
10,1145,1,61,45,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.350672,3.965366,4.165999,30.084959,0.376673,0.303365,0.386479
13,517,0,41,28,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.527501,3.610936,4.704285,1.020149,0.419662,0.431762,0.432416


### Prepare list of clients to target

In [ ]:
targets, forecast, df_targets = run_full_targeting_pipeline(predicted_revenues_df, top_frac=0.15)
print_targeting_summary(targets, forecast)

Stage 3,4: Assigning best offers...
Stage 5: Selecting top targets...
Stage 6: Calculating revenue forecast...

=== TARGETING SUMMARY ===
Total clients targeted: 96
Total expected revenue: $434.04
Average expected revenue per client: $2.78
Lift vs baseline targeting: 62.7%

Offer distribution:
  CL: 47 clients (49.0%)
  MF: 38 clients (39.6%)
  CC: 11 clients (11.5%)


In [ ]:
targets.head()

,Client,Best_Offer,Expected_Revenue,Age,Tenure
693,766,MF,9.357716,32,95
1408,498,CL,6.298910,21,176
962,1474,CL,6.245195,25,240
999,350,CL,6.086281,4,218
1450,731,CL,6.013221,21,181


In [ ]:
# save targets to targeted_clients.csv
df_targets[['Client', 'Best_Offer', 'Expected_Revenue']].to_csv('targeted_clients.csv', index=False)
